Copyright (C) Microsoft Corporation.  
see license file for details 

### Builds docker image for training (experimentation)

Runs basic tests and pushes image to dockerhub  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext dotenv

In [3]:
dotenv_file_path='./../not_shared/general.env'

In [4]:
import os

In [5]:
!pwd
!ls -l ./

/datadrive01/prj/AzureChestXRayNoAML/code
total 788
-rw-rw-r-- 1 loginvm022 loginvm022   7566 Nov 16 22:44 000_setup_azure_sa.ipynb
-rwxrwxr-x 1 loginvm022 loginvm022 473751 Nov 16 22:43 00_create_docker_image.html
-rwxrwxr-x 1 loginvm022 loginvm022  37116 Nov 16 22:44 010_create_training_docker_image.ipynb
drwxrwxr-x 5 loginvm022 loginvm022   4096 Nov 16 22:12 01_DataPrep
drwxrwxr-x 2 loginvm022 loginvm022   4096 Nov 14 11:30 02_Model
drwxrwxr-x 2 loginvm022 loginvm022   4096 Nov 14 11:53 docker_history
-rwxrwxr-x 1 loginvm022 loginvm022 256521 Nov 16 04:29 edit_python_files.html
-rwxrwxr-x 1 loginvm022 loginvm022   7152 Nov 16 04:30 edit_python_files.ipynb
drwxrwxr-x 3 loginvm022 loginvm022   4096 Nov 15 04:58 src


In [6]:
%dotenv $dotenv_file_path

# Your docker login and image repository name
docker_login = os.getenv('docker_login') 
image_tag = os.getenv('image_tag')
image_version = os.getenv('image_version')

docker_login
image_tag
image_version

'georgedockeraccount'

'/chestxray-no-aml-gpu-root-account'

'1.0.4'

In [7]:
%dotenv
os.getenv('image_version')
dockerfile_name = 'dockerfile'+ '_' + os.getenv('image_version')

'1.0.4'

In [8]:
%%writefile ./../docker/$dockerfile_name

FROM nvidia/cuda:10.0-cudnn7-devel
# FROM nvidia/cuda:9.2-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget \
      ssh \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh        

# Install AzCopy        
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.3.2:$PATH

# Install Python packages and keras
# ENV NB_USER keras
# ENV NB_UID 1000

# RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
#     chown $NB_USER $CONDA_DIR -R && \
#     mkdir -p /src && \
#     chown $NB_USER /src

# USER $NB_USER
RUN mkdir -p /sr

ARG python_version=3.6

RUN conda config --append channels conda-forge
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      tensorflow-gpu==1.14 \
      python-dotenv \
      papermill[azure] &&  \
    conda install \
      bcolz \
      h5py \
      imageio \
      matplotlib \
      mkl \
      numpy \
      nose \
      notebook \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      scikit-learn \
      scikit-image \
      six \
      theano \
      mkdocs \
      tqdm \
      && \
    conda install pytorch torchvision cudatoolkit=9.2 -c pytorch && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    conda clean -yt

# TF 1.13:cuda 10.0, cudnn 7 
# RUN conda install -y python=${python_version} && \
#     pip install --upgrade pip && \
#     pip install \
#       sklearn_pandas \
#       scikit-image \
#       tensorflow-gpu==1.14 \
#       tensorflow-tensorboard \
#       matplotlib \
#       numpy && \
#     conda install \
#       bcolz \
#       h5py \
#       matplotlib \
#       mkl \
#       nose \
#       notebook \
#       opencv \
#       Pillow \
#       pandas \
#       pydot \
#       pygpu \
#       pyyaml \
#       requests \
#       scikit-learn \
#       six \
#       tqdm \
#       pytorch torchvision cudatoolkit=9.0 -c pytorch && \
#     conda install -c conda-forge imageio && \
#     git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
#     pip install git+git://github.com/keras-team/keras.git && \
#     pip install git+https://github.com/aleju/imgaug && \
#     pip install git+https://www.github.com/keras-team/keras-contrib.git && \
#     conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile_1.0.4


In [9]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':' + os.getenv('image_version')
docker_file_location = os.path.join(*(['.', '..','docker', dockerfile_name]))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.4'

'./../docker/dockerfile_1.0.4'

In [10]:
!mkdir -p ./docker_history
!cp $docker_file_location ./docker_history/

In [11]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.4 -f ./../docker/dockerfile_1.0.4 .
Sending build context to Docker daemon  5.294MB
Step 1/16 : FROM nvidia/cuda:10.0-cudnn7-devel
 ---> eaf2ceb9de1a
Step 2/16 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 44abab7bfae5
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget       ssh       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> c5fb82908d50
Step 4/16 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> d4d519c56f2b
Step 5/16 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> 31aa9364de4c
Step 6/16 : RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh &&     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.1

In [12]:
cli_command_inside_container=' /bin/bash -c "conda env list; pwd; ls -l /workspace ; ' + \
    'pip3 list | grep tensorflow;' + \
    'python -c \'import tensorflow as tf; ' +\
    'from tensorflow.python.client import device_lib; ' +\
    'print(tf.__version__);' +\
    'gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1);' +\
    'sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options));' +\
    'local_device_protos = device_lib.list_local_devices(); ' +\
    'print( [x.name for x in local_device_protos if x.device_type == \\"GPU\\"]); ' +\
    'sess.close() ' +\
    '\';' +\
    '"'



cli_command_inside_container

cli_command='docker run --gpus all -it --rm  --name chestxray-no-aml-gpu_docker_container ' + \
'-v $(pwd):/workspace:rw ' + \
docker_image_name + \
cli_command_inside_container

cli_command
!$cli_command

' /bin/bash -c "conda env list; pwd; ls -l /workspace ; pip3 list | grep tensorflow;python -c \'import tensorflow as tf; from tensorflow.python.client import device_lib; print(tf.__version__);gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1);sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options));local_device_protos = device_lib.list_local_devices(); print( [x.name for x in local_device_protos if x.device_type == \\"GPU\\"]); sess.close() \';"'

'docker run --gpus all -it --rm  --name chestxray-no-aml-gpu_docker_container -v $(pwd):/workspace:rw georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.4 /bin/bash -c "conda env list; pwd; ls -l /workspace ; pip3 list | grep tensorflow;python -c \'import tensorflow as tf; from tensorflow.python.client import device_lib; print(tf.__version__);gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1);sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options));local_device_protos = device_lib.list_local_devices(); print( [x.name for x in local_device_protos if x.device_type == \\"GPU\\"]); sess.close() \';"'

# conda environments:
#
base                  *  /opt/conda

/src
total 788
-rw-rw-r-- 1 1003 1003   7566 Nov 16 22:44 000_setup_azure_sa.ipynb
-rwxrwxr-x 1 1003 1003 473751 Nov 16 22:43 00_create_docker_image.html
-rwxrwxr-x 1 1003 1003  37116 Nov 16 22:44 010_create_training_docker_image.ipynb
drwxrwxr-x 5 1003 1003   4096 Nov 16 22:12 01_DataPrep
drwxrwxr-x 2 1003 1003   4096 Nov 14 11:30 02_Model
drwxrwxr-x 2 1003 1003   4096 Nov 14 11:53 docker_history
-rwxrwxr-x 1 1003 1003 256521 Nov 16 04:29 edit_python_files.html
-rwxrwxr-x 1 1003 1003   7152 Nov 16 04:30 edit_python_files.ipynb
drwxrwxr-x 3 1003 1003   4096 Nov 15 04:58 src
tensorflow-estimator   1.14.0      
tensorflow-gpu         1.14.0      
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8,

2019-11-16 22:44:37.683383: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2019-11-16 22:44:37.683407: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10.0
2019-11-16 22:44:37.683429: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcurand.so.10.0
2019-11-16 22:44:37.683449: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcusolver.so.10.0
2019-11-16 22:44:37.683471: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcusparse.so.10.0
2019-11-16 22:44:37.683495: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7
2019-11-16 22:44:37.685245: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1763] Adding visible gpu devices: 0, 1
2019-11

In [13]:
cli_command = 'docker login  --username '+ os.getenv('docker_login') +' --password '+os.getenv('docker_password')
# cli_command
!$cli_command

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/loginvm022/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [14]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/chestxray-no-aml-gpu-root-account]

0486be30: Preparing 
3da388b2: Preparing 
8903c2e8: Preparing 
b4fea62e: Preparing 
5a905024: Preparing 
5f137119: Preparing 
3ee3c5e1: Preparing 
d5e86d56: Preparing 
fa18caaf: Preparing 
d8b4edec: Preparing 
bdc0b45f: Preparing 
de7ea906: Preparing 
7a55184c: Preparing 
5f137119: Waiting g 
13bce073: Preparing 
e43028b3: Preparing 
f3abed5f: Layer already exists 1.0.4: digest: sha256:91f9efe779740f75ad626b94b6bb25c96d186eac9e53dd3b8bef61cdcf602cae size: 3891


In [15]:
!jupyter nbconvert --to html 010_create_training_docker_image

[NbConvertApp] Converting notebook 010_create_training_docker_image.ipynb to html
[NbConvertApp] Writing 290232 bytes to 010_create_training_docker_image.html
